In [ ]:
# default_exp core

# SSUDock

> A super and simple package for docking study.

In [ ]:
#hide
from nbdev.showdoc import *

The python class for doing docking analysis

In [ ]:
#export
import shutil
import warnings
#from utils.utils import get, PDBParse, file_search
#from visual.MolView import *
#from dock.dock import *
#from dock.DockingTools import *
#from dock.ar2a import *
#from visual.DVisualize import *
import subprocess
from glob import glob
from rdkit.Chem.rdMolTransforms import ComputeCentroid
import pandas as pd
import numpy as np
from rich.table import Table
from rich.console import Console
import rich
from rdkit import Chem
from os.path import join, splitext
from collections import Counter
import os
import re
__date__ = "28-Sept-2021 10:34:51"
__author__ = "ssu"
__version__ = "0.0.2"
print(f"Last updated: {__date__ }")


# from rich import print

console = Console()
blue_console = Console(style="white on blue")
note_console = Console(style="white on red")


class SSUDock():
    """Creates a Project in Optimizing Scores and Docking
    Args:
        *args: Description
        **kwargs: Description
    """

    AA = None
    BASE_DIR = os.path.join(os.getcwd(), "DATA")

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.AA = [
            "ALA",
            "ARG",
            "ASN",
            "ASP",
            "CYS",
            "GLN",
            "GLU",
            "GLY",
            "HIS",
            "ILE",
            "LEU",
            "LYS",
            "MET",
            "PHE",
            "PRO",
            "SER",
            "THR",
            "TRP",
            "TYR",
            "VAL",
        ]
        SSUDock.AA = self.AA
        self.BASE_DIR = kwargs.get("path", SSUDock.BASE_DIR)
        for arg in args:
            self.BASE_DIR = arg
        self.BASE_DIR = os.path.join(os.getcwd(), self.BASE_DIR)
        if not os.path.exists(self.BASE_DIR):
            os.makedirs(self.BASE_DIR)
        console.print(f"Project Root set at {self.BASE_DIR}")

    def SetFolder(self, *args, **kwargs):
        actual_cwd = os.getcwd()
        self.BASE_DIR = kwargs.get("path", os.getcwd())
        for i in args:
            self.BASE_DIR = i
        if self.BASE_DIR == ".":
            return console.print(f"Project Base Directory: {actual_cwd}")
        if actual_cwd != self.BASE_DIR:
            try:
                os.chdir(self.BASE_DIR)
                working_dir = self.BASE_DIR
            except Exception as err:
                working_dir = f"{actual_cwd}/{self.BASE_DIR}"
                os.chdir(working_dir)
        console.print(f"Project Base Directory: {working_dir}")

    def ProjectTree(self, *args, **kwargs):
        path = os.getcwd() if self.BASE_DIR is None else self.BASE_DIR
        verbose = kwargs.get("verbose")
        self.directory_tree(path, verbose=verbose)

    def __actual_dir_name(self, path, root=None):
        """helper function for directory tree generation"""
        if root is not None:
            path = os.path.join(root, path)
        result = os.path.basename(path)
        if os.path.islink(path):
            realpath = os.readlink(path)
            result = f"{os.path.basename(path)} -> {realpath}"
        return result

    # =============================================================================
    def directory_tree(self, startpath, verbose=True, depth=-1):
        """Tree view of the project directory tree"
        directory_tree(path)
        """
        supported_file_format = {
            "txt",
            "pdb",
            "pdbqt",
            "sdf",
            "csv",
            "excel",
            "pickle",
        }
        console.print(
            f"Supported File Format :{supported_file_format}",
            style="bold green",
        )
        table = self.__create_table("bold magenta", "File Type", "Total Files")
        c = Counter(
            [
                splitext(i)[1][1:]
                for i in glob(join(startpath, "**"), recursive=True)
                if splitext(i)[1][1:] in supported_file_format
            ]
        )
        console.print("============Details of files====================")
        for ext, count in c.most_common():
            table.add_row(
                f'[bold green]{ext}[/bold green]', f'[red]{count}[/red]')
        console.print(table)
        if verbose:
            console.print("============Directory Tree====================")
            prefix = 0
            if startpath != "/":
                if startpath.endswith("/"):
                    startpath = startpath[:-1]
                prefix = len(startpath)
            for root, dirs, files in os.walk(startpath):
                level = root[prefix:].count(os.sep)
                if depth > -1 and level > depth:
                    continue
                indent = subindent = ""
                if level > 0:
                    indent = "|   " * (level - 1) + "|-- "
                subindent = "|   " * (level) + "|-- "
                print(
                    f"{indent}📂{self.__actual_dir_name(root)}/"
                )  # print dir only if symbolic link; otherwise, will be printed as root
                for d in dirs:
                    if not d.startswith(".") and os.path.islink(
                        os.path.join(root, d)
                    ):
                        print(
                            f"{subindent}📃{self.__actual_dir_name(d, root=root)}"
                        )
                for f in files:
                    _format = f.rsplit(".")[-1]
                    if _format in supported_file_format:
                        print(
                            f"{subindent}📃{self.__actual_dir_name(f, root=root)}"
                        )

    # =============================================================================
    def __receptor_contents_print(
        self, receptor, receptor_content, show=True, data=False
    ):
        number_of_residues = []
        residues = []
        number_of_membrane_molecules = []
        membrane = []
        water_molecule = []
        number_of_water_molecule = 0
        present_ions = []
        number_of_chains = []
        number_of_ligands = []
        number_of_ligands_atoms = 0
        ligand = []
        ions = []

        for index, line in enumerate(receptor_content):
            if line.startswith("ATOM"):
                if (
                    line[17:20] in self.AA
                    or line.split()[-1] == "PROA"
                    or line[21:22].strip() == "P"
                ):
                    number_of_chains.append(line[21])
                    number_of_residues.append(line[22:26])
                    residues.append(line)
                elif line.split()[-1] == "MEMB" or line[21:22].strip() == "M":
                    number_of_membrane_molecules.append(line[22:26])
                    membrane.append(line)
                elif (
                    line.split()[-1] == "TIP3"
                    or line[17:20] == "HOH"
                    or line[21:22].strip() == "T"
                ):
                    number_of_water_molecule += 1
                    water_molecule.append(line)
                elif (
                    line.split()[-1] == "HETA"
                    or line[72:76].strip() == "HETA"
                    or line[17:20] == "LIG"
                ):
                    number_of_ligands.append(line.split()[3])
                    ligand.append(line)
                else:
                    present_ions.append(line[17:20])
                    ions.append(line)
            elif line.startswith("HETATM"):
                if line[17:20] == "HOH":
                    number_of_water_molecule += 1
                    water_molecule.append(line)
                elif len(line[17:20].strip()) < 3:
                    present_ions.append(line[17:20])
                    ions.append(line)
                elif len(line[17:20].strip()) == 3:
                    # number_of_ligands.append(line.split()[3])
                    number_of_ligands.append(line[21])
                    number_of_ligands_atoms += 1
                    ligand.append(line)
        if not present_ions:
            max_number_of_single_ions = 0
        else:
            max_number_of_single_ions = max(
                present_ions, key=present_ions.count
            )
        types_of_ions = set(present_ions) if present_ions else 0
        _number_of_membrane_molecules = (
            number_of_membrane_molecules[-1]
            if len(number_of_membrane_molecules) > 1 and not None
            else 0
        )

        def style(num, text, can_return, data=data):
            if data:
                return f"[bold red]{can_return}[{num}][/][bold green]{text}:[/]"
            else:
                return f"[bold green]{text}:[/]"

        table = self.__create_table(
            "bold blue", f"{'DATA' if data else 'Complex'}", "Counts"
        )
        table.add_row(
            f"{style('!', 'Chain', 'x', data=data)}",
            f" {len(set(number_of_chains))}",
        )

        try:
            table.add_row(
                f"{style('!', 'Number of ligand atoms', 'x', data=data)}",
                f"{number_of_ligands.count(max(number_of_ligands, key=number_of_ligands.count))}",
            )
        except ValueError:
            table.add_row(
                f"{style('!', 'Number of ligand atoms', 'x', data=data)}", "0"
            )

        try:
            table.add_row(
                f"{style('0', 'Protein residues', 'R', data=data)}",
                f"{number_of_residues[-1]}",
            )
        except Exception:
            table.add_row(
                f"{style('0', 'Proteion residues', 'R', data=data)}", "0"
            )

        try:
            table.add_row(
                f"{style('1', 'Lipid molecules', 'R', data=data)}",
                f"{_number_of_membrane_molecules}",
            )
        except ValueError:
            table.add_row(
                f"{style('1', 'Lipid molecules', 'R', data=data)}", "0"
            )

        try:
            table.add_row(
                f"{style('2', 'Water molecules', 'R', data=data)}",
                f" {number_of_water_molecule}",
            )
        except ValueError:
            table.add_row(
                f"{style('2', 'Water molecules', 'R', data=data)}", "0"
            )
        try:
            table.add_row(
                f"{style('3', 'Ions', 'R', data=data)}", f"{len(present_ions)}"
            )
            table.add_row(
                f"{style('3', 'Ion types', 'R',data=data)}", f"{types_of_ions}"
            )
        except ValueError:
            table.add_row(f"{style('3', 'Ions', 'R',data=data)}", "0")
            table.add_row(f"{style('3', 'Ion types', 'R',data=data)}", "None")
        table.add_row(
            f"{style('4', 'Ligands', 'R',data=data)}",
            f"{len(set(number_of_ligands))}",
        )

        try:
            if self.box and data is False:
                table.add_row(
                    f"{style('0', 'Box Center', 'Box')}", f"{self.box[0]}"
                )
                table.add_row(
                    f"{style('1', 'Box Size', 'Box')}", f"{self.box[1]}"
                )
        except Exception:
            pass

        if show:
            # console.print(f"\nFor[bold red] {receptor}[/]:")
            console.print(table)
        else:
            return residues, membrane, water_molecule, ions, ligand

    # =============================================================================

    def separate_save(self, content, filename):
        if not os.path.exists(self.BASE_DIR):
            os.makedirs(self.BASE_DIR)
        temp_path = os.path.join(self.BASE_DIR, filename)
        with open(temp_path, "w+") as fw:
            for line in content:
                print(line, end="", file=fw)
        return temp_path

    def LoadReceptor(
        self,
        *args,
        info=False,
        native=True,
        inplace=True,
        data=True,
        verbose=True,
        UNIVERSAL=False,
        **kwargs,
    ):
        found_what = kwargs.get("key", "Receptor")
        for i in args:
            receptor = i
        try:
            if os.path.exists(os.path.join(self.BASE_DIR, receptor)):
                receptor = os.path.abspath(
                    os.path.join(self.BASE_DIR, receptor)
                )
        except TypeError:
            pass

        if isinstance(receptor, list):
            receptor_content = receptor
        else:
            if not os.path.exists(receptor):
                try:
                    _receptor = file_search(type="pdb", target=receptor)
                    if len(_receptor) == 1:
                        console.print(
                            f"{found_what}: [bold]{_receptor[0]}[/bold]"
                        )
                        info = False
                        receptor = _receptor[0]
                    elif len(_receptor) > 1:
                        print(f"{found_what} {_receptor}")
                        _receptor_number = int(
                            input(
                                f"Which {found_what} do you like to select: ")
                        )
                        receptor = _receptor[_receptor_number]
                        console.print(f"Select {found_what} : {receptor}")
                    else:
                        print(f"No {found_what} found in Local directory")
                        if native:
                            _download = input(
                                f"Would you like to download the {receptor} from RCSB (Y/n)? "
                            )
                            confirm = ["yes", "y", "YES", "Y"]
                            if _download in confirm:
                                download_protein_msg = get(receptor)
                                # TODO path pass forward
                                console.print(download_protein_msg)
                                receptor = f"{self.BASE_DIR}/{receptor}.pdb"
                            else:
                                console.print(
                                    f"😞 {found_what}: [bold red]{receptor} [/]to process"
                                    " further.."
                                )
                except Exception as er:
                    print(er)
            if info is True:
                console.print(f"{found_what}: [bold]{receptor}[/bold]")
                info = False
            receptor_content = open(receptor, "r")
            receptor_content = receptor_content.readlines()
        if verbose:
            self.__receptor_contents_print(
                receptor, receptor_content, show=True, data=data
            )
        # console.print(self.receptor)
        #  changed
        (
            Rreceptor,
            Rlipid,
            Rwater,
            Rion,
            Rligand,
        ) = self.__receptor_contents_print(
            receptor, receptor_content, show=False, data=data
        )

        if UNIVERSAL:
            self.UNIVERSAL_PDB = receptor
            self.UNIVERSAL_Rreceptor = Rreceptor
            self.UNIVERSAL_Rlipid = Rlipid
            self.UNIVERSAL_Rwater = Rwater
            self.UNIVERSAL_Rion = Rion
            self.UNIVERSAL_Rligand = Rligand

        if native:
            self.__setattr__("receptor", receptor)
            if inplace:
                self.receptor = self.separate_save(
                    Rreceptor, "temp_receptor.pdb"
                )
            else:
                self.receptor = receptor
            if Rligand:
                self.ligand = self.separate_save(Rligand, "temp_ligand.pdb")
            if Rlipid:
                self.separate_save(Rwater, "temp_water.pdb")
                self.separate_save(Rion, "temp_ion.pdb")
                self.lipid = self.separate_save(Rlipid, "temp_Rlipid.pdb")

                # Rlipid.extend([x for x in Rwater])
                # Rlipid.extend([x for x in Rion])
                # self.lipid = self.separate_save(Rlipid, "temp_lipid.pdb") # commented TODO double print

        return (Rreceptor, Rlipid, Rwater, Rion, Rligand, receptor)

    def __create_table(self, header_style, arg1, arg2):
        result = Table(show_header=True, header_style=header_style)
        result.add_column(arg1, style="dim", width=40)
        result.add_column(arg2, justify="right")
        return result

    # =============================================================================

    def LoadLigand(
        self, ligand, center=False, verbose=False, centroid=False, native=True
    ):
        """Load Ligand into the project

        Args:
            ligand ([str]): Path of the small molecule
            native (bool, optional): [description]. Defaults to True.
            give_path (bool, optional): to get back path. Defaults to False.

        Returns:
            str: Retrun the path of the small molecules

        """
        if isinstance(ligand, list):
            if not os.path.exists(self.BASE_DIR):
                os.makedirs(f"{self.BASE_DIR}")
            with open(f"{self.BASE_DIR}/temp_ligand.pdb", "w+") as temp_lig:
                for i in ligand:
                    print(i, end="", file=temp_lig)
            ligand = f"{self.BASE_DIR}/temp_ligand.pdb"

        *_, _file_format = give_id(ligand)
        if _file_format.lower() == "sdf":
            inf = open(f"{ligand}", "rb")
            with Chem.ForwardSDMolSupplier(inf) as fsuppl:
                for mol in fsuppl:
                    if mol is None:
                        continue
                    console.print(f"{ligand} has {mol.GetNumAtoms()} atoms")
        elif _file_format.lower() == "pdb":
            mol = Chem.MolFromPDBFile(ligand, sanitize=False)
            if mol.GetNumAtoms() == 1:
                *_, lig = PDBParse(ligand)  # rdkit not able to parse properly
                Ligand_num = len(lig)
            else:
                Ligand_num = mol.GetNumAtoms()
            console.print(
                f"⚠️[bold green]Ligand Information~ [/]{ligand} has {Ligand_num} atoms ⚛️"
            )
        else:
            return "Unknown ligand file format"

        # print(Chem.MolToMolBlock(mol))

        if native:
            self.__setattr__("ligand", ligand)
            self.ligand = ligand
            self.ligand_export = mol
            self.Rligand = [x for x in Chem.MolToPDBBlock(mol).split("\n")]
        centroid = ComputeCentroid(mol.GetConformer())
        # for atom in (mol.GetAtoms()):
        # print(atom)
        x = Chem.MolToXYZBlock(mol)
        x = x.split("\n")
        del x[0]
        x = list(filter(None, x))
        xyz = [i.split()[1:] for i in x]
        # print(mol.GetAtomWithMaxX)

        xyz = list(filter(None, xyz))

        x_coordinate = [float(x[0]) for x in xyz]
        y_coordinate = [float(x[1]) for x in xyz]
        z_coordinate = [float(x[2]) for x in xyz]
        if verbose:
            console.print(
                f"Min X: {min(x_coordinate)}, Max X: {max(x_coordinate)}"
            )
            console.print(
                f"Min Y: {min(y_coordinate)}, Max Y: {max(y_coordinate)}"
            )
            console.print(
                f"Min Z: {min(z_coordinate)}, Max Z: {max(z_coordinate)}"
            )

        # if centroid:
        #    console.print(
        #        f"Centroid :{centroid.x:.4f}, {centroid.y:.4f}, {centroid.z:.4f}")
        #    #return [centroid.x, centroid.y, centroid.z]
        if center:
            return (
                centroid.x,
                centroid.y,
                centroid.z,
                min(x_coordinate),
                max(x_coordinate),
                min(y_coordinate),
                max(y_coordinate),
                min(z_coordinate),
                max(z_coordinate),
            )
        return mol

    # =============================================================================
    def SaveComplex(self, **kwargs):
        """Save s the state of project."""
        ligand = kwargs.get("lig", self.ligand)
        receptor = kwargs.get("pro", self.receptor)
        lipid = kwargs.get("lipid", self.lipid)
        out_file = kwargs.get("out", "complex_out.pdb")
        try:
            ligand_mol = Chem.MolToPDBBlock(ligand, flavor=32)
        except AttributeError:
            pass
        *_, structure_format = give_id(receptor)
        write_list = []
        try:
            if structure_format.lower() == "sdf":
                receptor_mol = Chem.MolFromMolFile(receptor, removeHs=False)
            elif structure_format.lower() == "pdb":
                receptor_mol, *_ = PDBParse(receptor)
            write_list.append(receptor_mol)
        except ValueError as er:
            console.print(f"{self.receptor} Error : {er}")
            blue_console.print("Not able to parse..")
        try:
            write_list.append(ligand_mol)
        except Exception as er:
            pass
        try:
            _prot, lipid_mol, _tip, _lig = PDBParse(lipid)
            write_list.append(lipid_mol)
        except Exception as er:
            blue_console.print(f"Cannot able to parse {lipid}.\n Error: {er}")
            blue_console.print(f"Not writing Lipid in the {out_file}")
        try:
            if self.Rwater:
                write_list.append(self.Rwater)
            if self.Rion:
                write_list.append(self.Rion)
        except ValueError as er:
            print(er)
        self.__write_to_file(write_list, out_file, check=True)

    def __write_to_file(self, content, filename, check=False):
        if check:
            count = 0
            _file, _format = filename.rsplit(".")
            while os.path.exists(filename):
                count += 1
                filename = f"{_file}_{count}.{_format}"
        with open(filename, "a+") as f:
            if isinstance(content, list):
                for _ in content:
                    for line in _:
                        print(line, end="", file=f)
            else:
                for line in content:
                    print(line, end="", file=f)
        console.print(f"{filename} Saved Successfully!", style="bold green")

    # =============================================================================
    def LoadPDB(self, *args, **kwargs):

        receptor = kwargs.get("receptor")
        ligand = kwargs.get("ligand")
        lipid = kwargs.get("lipid")
        verbose = kwargs.get("verbose", True)
        for arg in args:
            receptor = arg
            if os.path.exists(receptor):
                receptor = os.path.abspath(receptor)
        if receptor:
            (
                Rreceptor,
                Rlipid,
                Rwater,
                Rion,
                Rligand,
                Rrecp_path,
            ) = self.LoadReceptor(
                receptor,
                native=False,
                verbose=verbose,
                UNIVERSAL=True,
                data=True,
            )
            return Rreceptor, Rlipid, Rwater, Rion, Rligand, Rrecp_path
        if ligand:
            if os.path.exists(ligand):
                ligand = os.path.abspath(ligand)
            self.LoadLigand(ligand)
        if lipid:
            if os.path.exists(lipid):
                lipid = os.path.abspath(lipid)

            self.lipid, self.Rlipid = self.LoadLipid(lipid, verbose=verbose)

    def LoadSDF(self, ligand):
        if os.path.exists(ligand):
            ligand = os.path.abspath(ligand)
        self.__setattr__("ligand", ligand)
        self.LoadLigand(ligand)

    def __setattr__(self, name, value):
        object.__setattr__(self, name, value)

    # =============================================================================

    def ShowData(self):
        return self.__str__()

    @property
    def receptor_content(self):
        note_console.print("Note: Content from Receptor")
        return self.Rreceptor

    @property
    def R0(self):
        note_console.print("Note: Content from Receptor")
        return self.Rreceptor

    @property
    def lipid_content(self):
        note_console.print("Note: Content from Receptor")
        return self.Rlipid

    @property
    def R1(self):
        note_console.print("Note: Content from Receptor")
        return self.Rlipid

    @property
    def water_content(self):
        note_console.print("Note: Content from Receptor")
        return self.Rwater

    @property
    def R2(self):
        note_console.print("Note: Content from Receptor")
        return self.Rwater

    @property
    def ion_content(self):
        note_console.print("Note: Content from Receptor")
        return self.Rion

    @property
    def R3(self):
        note_console.print("Note: Content from Receptor")
        return self.Rion

    @property
    def ligand_content(self):
        note_console.print("Note: Content from Receptor")
        return self.Rligand

    @property
    def R4(self):
        note_console.print("Note: Content from Receptor")
        return self.Rligand

    @property
    def box_dimension(self):
        return self.box

    @property
    def ShowOriginal(self):
        return self.Show(
            self.UNIVERSAL_Rreceptor,
            self.UNIVERSAL_Rlipid,
            self.UNIVERSAL_Rwater,
            self.UNIVERSAL_Rion,
            self.UNIVERSAL_Rligand,
        )

    @property
    def poses(self):
        try:
            if self.conformers:
                console.print(
                    f"Total docked poses : {len(self.conformers)}",
                    style="bold blue",
                )
                return self.conformers
            else:
                return "No poses"
        except AttributeError:
            return "No poses found."

    # #################################################################################################
    def monitor(
        self,
        type="pdb",
        target="*_output_*",
        back=False,
        specific=None,
        plot=True,
        save=False,
    ):
        try:
            self.smina_output_monitor = file_search(
                type=type, target=target, specific=specific
            )
            _ = smina_monitor(self.smina_output_monitor, plot=plot, save=save)
            if back:
                return _

        except ValueError:
            print(
                "No file found to display information."
                "Please specify the output folder of smina using 'specific='DATA''."
            )

    def split_ligands(self, type="pdb"):
        conformer_split(self.smina_output_monitor, type)

    def combine_conformers(
        self,
        conformers_file_list=None,
        new_file="combines",
        file_format="sdf",
        solver="Default",
    ):

        if conformers_file_list is None and self.conformers is not None:
            conformers_file_list = self.conformers
        count = 0
        name = f"{self.BASE_DIR}/{new_file}_{count}.{file_format}"
        while os.path.exists(name):
            count += 1
            name = f"{self.BASE_DIR}/{new_file}_{count}.{file_format}"

        if solver == "Default":
            with open(f"{self.BASE_DIR}/{name}", "w+") as out:
                for file in conformers_file_list:
                    with open(file) as infile:
                        out.write(infile.read())
                    out.write("\n")
        elif solver == "obabel":
            command = f"/share/anaconda3/bin/obabel {' '.join([str(elem) for elem in conformers_file_list])} -O {name}"
            _ = subprocess.run(
                command, shell=True, capture_output=True, text=True
            )
            console.print(_.stdout)
        console.print(f"{name} saved successfully")
        return name

    # =============================================================================

    def __call__(self):
        raise TypeError(
            "Project must be accessed through" '"instance=SSUDock()".'
        )

    def __str__(self):
        self.LoadReceptor(
            self.UNIVERSAL_PDB, data=True, native=False, inplace=False
        )
        _temp_path = self.Save(
            "temp_ComplexData.pdb", verbose=False, path=True, rename=False
        )
        self.LoadReceptor(_temp_path, data=False, native=False, inplace=False)
        # os.remove(_temp_path)

        # read all varibel write to a file anmd reload to receptr wil complex informations

    def AutoBox(
        self,
        *args,
        show=False,
        inplace=True,
        box_inplace=False,
        verbose=True,
        verbose_minmax=True,
        **kwargs,
    ):
        ligand = args[0]
        if inplace:
            native = True
        else:
            native = False

        (
            box_center_x,
            box_center_y,
            box_center_z,
            minx_coordinate,
            maxx_coordinate,
            miny_coordinate,
            maxy_coordinate,
            minz_coordinate,
            maxz_coordinate,
        ) = self.LoadLigand(ligand, center=True, native=native, verbose=False)
        try:
            padding = args[1]
        except Exception:
            padding = 20

        def both_same_int(a, b):
            if a <= 0 and b >= 0 or a >= 0 and b <= 0:
                return abs(a) + abs(b)
            elif a <= 0 and b <= 0:
                return min(a, b)
            elif a >= 0 and b >= 0:
                return max(a, b)
            else:
                return a

        x_xtra_pad = both_same_int(maxx_coordinate, minx_coordinate)
        y_xtra_pad = both_same_int(maxy_coordinate, miny_coordinate)
        z_xtra_pad = both_same_int(maxz_coordinate, minz_coordinate)
        box_center_x = round(box_center_x, 3)
        box_center_y = round(box_center_y, 3)
        box_center_z = round(box_center_z, 3)
        box_size_x = abs(
            round((x_xtra_pad + box_center_x + padding), 3))  # std
        box_size_y = abs(round((y_xtra_pad + box_center_y + padding), 3))
        box_size_z = abs(round((z_xtra_pad + box_center_z + padding), 3))

        box = [
            [box_center_x, box_center_y, box_center_z],
            [box_size_x, box_size_y, box_size_z],
        ]

        table = self.__create_table("bold red", "Box Information", "Points")
        if verbose_minmax:
            table.add_row("[bold blue u]ligand min [/]")
            table.add_row(
                "[bold red]x[!][!][/][bold green]ligand_min_x:[/]",
                f" {minx_coordinate}",
            )
            table.add_row(
                "[bold red]x[!][!][/][bold green]ligand_min_y:[/]",
                f" {miny_coordinate}",
            )
            table.add_row(
                "[bold red]x[!][!][/][bold green]ligand_min_z:[/]",
                f" {minz_coordinate}",
            )
            table.add_row()
            table.add_row("[bold blue u]ligand max [/]")
            table.add_row(
                "[bold red]x[!][!][/][bold green]ligand_max_x:[/]",
                f" {maxx_coordinate}",
            )
            table.add_row(
                "[bold red]x[!][!][/][bold green]ligand_max_y:[/]",
                f" {maxy_coordinate}",
            )
            table.add_row(
                "[bold red]x[!][!][/][bold green]ligand_max_z:[/]",
                f" {maxz_coordinate}",
            )
            table.add_row()
        table.add_row("[bold blue u]box centers[/]")
        table.add_row(
            "[bold red]x[0][0][/][bold green]box_center_x:[/]",
            f" {box_center_x}",
        )
        table.add_row(
            "[bold red]x[0][1][/][bold green]box_center_y:[/]",
            f" {box_center_y}",
        )
        table.add_row(
            "[bold red]x[0][2][/][bold green]box_center_z:[/]",
            f" {box_center_z}",
        )
        table.add_row()
        table.add_row("[bold blue u]box size[/]")
        table.add_row(
            "[bold red]x[1][0][/][bold green]box_size_x:[/]", f" {box_size_x}"
        )
        table.add_row(
            "[bold red]x[1][1][/][bold green]box_size_y:[/]", f" {box_size_y}"
        )
        table.add_row(
            "[bold red]x[1][2][/][bold green]box_size_z:[/]", f" {box_size_z}"
        )
        if verbose:
            console.print(table)
        if box_inplace:
            self.__setattr__("box", box)
            self.__setattr__("box_center_x", box_center_x)
            self.__setattr__("box_center_y", box_center_y)
            self.__setattr__("box_center_z", box_center_z)
            self.__setattr__("box_size_x", box_size_x)
            self.__setattr__("box_size_y", box_size_y)
            self.__setattr__("box_size_z", box_size_z)
            self.LoadBox(box[0], box[1])
        if show:
            self.Show
        return box

    def Show(self, *args, show=True, **kwargs):
        """[ protein, ligand, lipid, box]"""

        if len(args) < 1:
            return "Nothing to Show. Add arguments to show. eg~ x.Show(R[0])"
        out = kwargs.get("out", "complex_out.pdb")
        save = kwargs.get("save", False)
        AA = SSUDock.AA

        def add_model(vobj, content, AA):
            if all(isinstance(elem, list) for elem in content):
                center_arg, size_arg = content[0], content[-1]
                box_center_x = center_arg[0]
                box_center_y = center_arg[1]
                box_center_z = center_arg[2]
                box_size_x = size_arg[0]
                box_size_y = size_arg[1]
                box_size_z = size_arg[2]
                vobj.addBox(
                    {
                        "center": {
                            "x": box_center_x,
                            "y": box_center_y,
                            "z": box_center_z,
                        },
                        "dimensions": {
                            "w": box_size_x,
                            "h": box_size_y,
                            "d": box_size_z,
                        },
                        "color": "blue",
                        "opacity": 0.5,
                    }
                )
                return vobj

            def check_file(file):
                return bool(isinstance(file, str) and os.path.exists(file))

            count = 0
            temp_name = f"temp_name_{count}.pdb"
            # if not os.path.exists("./.temp/"):
            #    os.makedirs("./.temp/")
            while os.path.exists(os.path.join(SSUDock.BASE_DIR, temp_name)):
                count += 1
                temp_name = f"temp_name_{count}.pdb"
            if check_file(content):
                _temp_path = content
                content = open(_temp_path, "r")
                content = content.readlines()
            else:
                _temp_path = temp_write(content, temp_name, special=True)
            vobj.addModel(open(_temp_path).read())
            style = mol_identifier(content, AA)
            vobj.setStyle({"model": count}, style)
            return vobj

        def mol_identifier(content, AA):
            for line in content:
                if line.startswith("ATOM"):
                    if (
                        line[17:20] in AA
                        or line.split()[-1] == "PROA"
                        or line[21:22].strip() == "P"
                    ):
                        return {"cartoon": {"color": "spectrum"}}
                    elif (
                        line.split(
                        )[-1] == "MEMB" or line[21:22].strip() == "M"
                    ):
                        return {"line": {}}
                    elif (
                        line.split()[-1] == "TIP3"
                        or line[17:20] == "HOH"
                        or line[21:22].strip() == "T"
                    ):
                        return {"sphere": {"opacity": 0.9, "color": "red"}}
                    elif (
                        line.split()[-1] == "HETA"
                        or line[72:76].strip() == "HETA"
                        or line[17:20] == "LIG"
                    ):
                        return {"stick": {}}
                    else:
                        return {
                            "sphere": {
                                "opacity": 0.9,
                                "scale": 0.7,
                                "colorscheme": "ssPyMOL",
                            }
                        }
                elif line.startswith("HETATM"):
                    if line[17:20] == "HOH":
                        return {"sphere": {"opacity": 0.9, "color": "blue"}}
                    elif len(line[17:20].strip()) < 3:
                        return {
                            "sphere": {
                                "opacity": 0.9,
                                "scale": 0.7,
                                "colorscheme": "ssPyMOL",
                            }
                        }
                    elif len(line[17:20].strip()) == 3:
                        return {"stick": {}}

        vobj = py3Dmol.view(width=800, height=600)
        write = []
        for arg in args:
            vobj = add_model(vobj, arg, AA)
            write.append(arg)
        vobj.zoomTo()
        if save:
            out_name, out_format = out.rsplit(".")
            count = 0
            while os.path.exists(out):
                count += 1
                out = f"{out_name}_{count}.{out_format}"
            with open(out, "w+") as wr:
                for _ in write:
                    for line in _:
                        print(line, end="", file=wr)
            console.print(f"{out} Successfully Saved !", style="bold green")
        del_command = f"rm -rf {SSUDock.BASE_DIR}/temp_name*"
        subprocess.run(del_command, shell=True)
        if show:
            return vobj.show()

    # =============================================================================

    def MakeComplex(self, *molecules, charm_gui=True, **kwargs):
        """Make Complex with information passed as argmunent and stores in instance."""
        out = kwargs.get("out", "complex_out.pdb")
        save = kwargs.get("save", False)
        AA = SSUDock.AA
        lipid = kwargs.get("l", None)
        ligand = kwargs.get("s", None)
        receptor = kwargs.get("r", None)
        box = kwargs.get("box", None)
        ions = kwargs.get("ions", None)
        water = kwargs.get("water", None)

        for mol in molecules:
            if all(isinstance(elem, list) for elem in mol):
                box = mol
            else:
                what_identified = molecule_identifier(mol, AA)
                if what_identified == "ion":
                    ions = mol
                elif what_identified == "ligand":
                    ligand = mol
                elif what_identified == "protein":
                    receptor = mol
                elif what_identified == "water":
                    water = mol
                else:
                    lipid = mol
        try:
            if receptor:
                self.__setattr__("receptor", receptor)
                (
                    self.Rreceptor,
                    self.Rlipid,
                    self.Rwater,
                    self.Rion,
                    self.Rligand,
                    self.Rrecp_path,
                ) = self.LoadReceptor(
                    receptor, navtive=True, verbose=False, data=False
                )
            else:
                self.__setattr__("receptor", None)
            if lipid:
                self.__setattr__("lipid", lipid)
                self.LoadLipid(
                    lipid,
                    info=False,
                    native=False,
                    inplace=False,
                    verbose=False,
                    data=False,
                )
            else:
                self.__setattr__("lipid", None)
            if box:
                self.__setattr__("box", box)
                self.LoadBox(box[0], box[1], inplace=True, box_inplace=True)
            else:
                self.__setattr__("box", None)
            if ligand:
                self.__setattr__("ligand", ligand)
                self.LoadLigand(ligand)
            else:
                self.__setattr__("ligand", None)
        except Exception:
            pass
        if save:
            count = 0
            _name, _out_format = out.split(".")
            while os.path.exists(out):
                count += 1
                out = f"{_name}_{count}.{_out_format}"
            with open(os.path.join(self.BASE_DIR, out), "w+") as complex:
                try:
                    with open(receptor, "r") as prot:
                        for line in prot:
                            print(line, end="", file=complex)
                except Exception:
                    # print(er)
                    pass
                try:
                    with open(lipid, "r") as lip:
                        for line in lip:
                            print(line, end="", file=complex)
                except Exception:
                    # print(er)
                    pass
                try:
                    with open(ligand, "r") as lig:
                        for line in lig:
                            print(line, end="", file=complex)
                except Exception:
                    # print(er)
                    pass
                if charm_gui is False:
                    try:
                        with open(water, "r") as wat:
                            for line in wat:
                                print(line, end="", file=complex)
                    except Exception:
                        # print(er)
                        pass
                    try:
                        with open(ions, "r") as ions_read:
                            for line in ions_read:
                                print(line, end="", file=complex)
                    except Exception:
                        # print(er)
                        pass
                console.print(f"{out} saved successfully", style="bold green")

    def rename_existing(self, infile, verbose=True):
        out_name, out_format = infile.rsplit(".")
        name = f"{out_name}.{out_format}"
        out = os.path.join(self.BASE_DIR, name)
        if not os.path.exists(os.path.dirname(out)):
            os.makedirs(os.path.dirname(out))
        old = out
        x = True
        if os.path.exists(old):
            count = 0
            while x:
                new = os.path.join(
                    self.BASE_DIR, f"{out_name}_old_{count}.{out_format}"
                )
                if os.path.exists(new):
                    count += 1
                else:
                    x = False
            os.rename(old, new)
            if verbose:
                note_console.print(
                    f"Renamed existing {os.path.basename(out)} to {os.path.basename(new)}"
                )
            # out = os.path.join(self.BASE_DIR, f"{out_name}_{count}.{out_format}")
        return out, out_format

    def Save(self, infile, verbose=True, path=False, rename=True):
        write = []
        if rename:
            out, out_format = self.rename_existing(infile, verbose=verbose)
        else:
            out_name, out_format = infile.rsplit(".")
            name = f"{out_name}.{out_format}"
            out = os.path.join(self.BASE_DIR, name)

        if self.receptor:
            if os.path.exists(self.receptor):
                receptor = open(self.receptor).read()
                write.append(receptor)
            else:
                write.append(self.receptor)
        if self.lipid:
            if isinstance(self.lipid, list):
                self.lipid = os.path.join(self.BASE_DIR, "temp_lipid.pdb")
            if os.path.exists(self.lipid):
                lipid = open(self.lipid).read()
                write.append(lipid)
            else:
                write.append(self.lipid)
        if self.ligand:
            if os.path.exists(self.ligand):
                ligand = open(self.ligand).read()
                write.append(ligand)
            else:
                write.append(self.ligand)

        with open(out, "w+") as write_out:
            for item in write:
                for line in item:
                    print(line, end="", file=write_out)
        if out_format != "pdb":
            mol = Chem.MolFromPDBFile(out)
            try:
                os.remove(out)
                print(Chem.MolToMolBlock(mol), file=open(out, "w+"))
            except Exception:
                raise warnings.warn("\nCOULD NOT CONVERT SOME LIPIDS TO SDF!!")
        if verbose:
            console.print(f"{out} Successfully Saved !")
        if path:
            return out

    #################################Docking#############################################

    def dock(
        self,
        r=None,
        l=None,
        sf=["vina"],
        box=None,
        score_only=False,
        inhouse=False,
        nomatch=True,
        queue=None,
        autobox=None,
        cpu=2,
        job_name=None,
        out=None,
        log=None,
        out_format="pdb",
        num_modes=10,
    ):
        try:
            if os.path.exists(os.path.join(self.BASE_DIR, r)):
                receptor = [os.path.join(self.BASE_DIR, r)]
        except TypeError:
            pass
        try:
            if os.path.exists(os.path.join(self.BASE_DIR, l)):
                ligand = [os.path.join(self.BASE_DIR, l)]
        except Exception as er:
            print(er)

        if inhouse:
            sf = receptor
            receptor = [self.receptor]
            ligand = [self.ligand]
        if box is None:
            box = self.box
        console.print(
            "[bold bright_magenta]Confirm Docking Information:[/]\n *************************",
            style="bold green",
        )
        console.print(
            f"\tReceptor : [bold deep_sky_blue1 u]{os.path.basename(receptor[0])}[/]\n"
            f"\tLigand : [bold blue u]{os.path.basename(ligand[0])}[/]\n"
            f"\tBox : [bold dark_magenta u]{box}[/]\n"
            f"\tScoring functions : [bold purple4 u]{sf}[/]\n"
        )
        if score_only:
            note_console.print("Docking Score Only Mode is Activated!!")
        if input("START DOCKING??: ").lower() in ["yes", "y"]:
            console.print(
                "***************************************\n", style="blue"
            )
            Dock.dock(
                self,
                receptor,
                ligand,
                sf,
                box,
                nomatch=nomatch,
                score_only=score_only,
                autobox=autobox,
                cpu=cpu,
                cluster=queue,
                job_name=job_name,
                out_format=out_format,
                out=out,
                log=log,
                num_modes=num_modes,
            )
            console.print(
                "***************************************\n", style="bold red"
            )
            _ = subprocess.run(
                "qstat", shell=True, capture_output=True, text=True
            )
            console.print(_.stdout, end="")
        else:
            note_console.print("No Docking performed!!")

    def ShowComplex(self, water=False, ions=False):
        show_list = []
        try:
            if self.receptor:
                R0 = open(self.receptor, "r")
                R0 = R0.readlines()
                show_list.append(R0)
        except AttributeError:
            pass
        try:
            if self.ligand:
                R4 = open(self.ligand, "r")
                R4 = R4.readlines()
                show_list.append(R4)
        except AttributeError:
            pass
        try:
            if self.box:
                show_list.append(self.box)
        except AttributeError:
            pass
        try:
            if self.lipid:
                try:
                    R1 = open(
                        os.path.join(self.BASE_DIR, "temp_Rlipid.pdb"), "r"
                    )
                except FileNotFoundError:
                    R1 = open(
                        os.path.join(self.BASE_DIR, "temp_lipid.pdb"), "r"
                    )  # TODO need to fix why Rlipid removed where
                R1 = R1.readlines()
                show_list.append(R1)
            if water:
                R2 = open(os.path.join(self.BASE_DIR, "temp_water.pdb"), "r")
                R2 = R2.readlines()
                show_list.append(R2)
            if ions:
                R3 = open(os.path.join(self.BASE_DIR, "temp_ion.pdb"), "r")
                R3 = R3.readlines()
                show_list.append(R3)
        except AttributeError:
            pass
        if show_list:
            self.Show(*show_list)
        return

    # utilities ================================================

    def rmsd_matrix(
        self,
        poses=None,
        poses_type="pdb",
        target="*_output_*",
        specific="poses",
        plot=True,
        save=False,
        annot=False,
        verbose=False,
        back=True,
        key="Latest_matrix",
        rmsd_save=True,
    ):

        if poses is None:
            _poses = file_search(
                type=poses_type, target=target, specific=specific
            )
        else:
            _poses = poses
        if _poses is None:
            return " Failed to find the poses.Try 'poses_type='pdb' to specify the type of poses or Try 'target='*_output_*'' to specify the poses for string  regrex match'"
        _df = rmsd_matrix(
            _poses,
            plot=plot,
            save=save,
            annot=annot,
            fig_size=(10, 8),
            verbose=verbose,
            key=key,
            rmsd_save=rmsd_save,
            BASE_DIR=self.BASE_DIR,
        )
        self.conformers = _poses
        if back:
            return _df

    def parse_score(self, filename):
        for count, i in enumerate(filename):
            term_values = []
            with open(i, "r") as rd:
                for line in rd:
                    x = "## Name"
                    if line.startswith(x):
                        cols = line.replace(x, "").split()
                        cols.insert(0, "name")
                        cols.insert(len(cols), "affinity")
                    elif line.startswith("Affinity:"):
                        _affinity = line.split()[1]
                    elif line.startswith("##"):
                        term_values.append(line.split()[1:])
                        term_values[-1].extend([_affinity])
            temp_df = pd.DataFrame(term_values, columns=cols)
            temp_df["name"] = [os.path.basename(x) for x in temp_df.name]

            return temp_df

    def state(
        self, data_frame, inactive="inactive", active="active", value=[1, 0]
    ):
        content = [active, inactive]
        value = value
        content_list = [
            data_frame.name.str.lower().str.contains(con) for con in content
        ]
        data_frame["State"] = np.select(content_list, value)
        return data_frame


def temp_write(x, name, special=False, clean=False):
    try:
        if os.path.exists(x):
            # print("File exists.. so parsing..",sep="",  flush=True)
            return os.path.abspath(os.path.join(os.getcwd(), x))
    except Exception:
        pass
    if not os.path.exists(SSUDock.BASE_DIR):
        os.makedirs(SSUDock.BASE_DIR)
    temp_path = os.path.join(SSUDock.BASE_DIR, name)
    if isinstance(x, list):
        with open(temp_path, "w+") as t:
            for _ in x:
                print(_, end="", file=t)
        return temp_path
    if special:
        if isinstance(x, Chem.rdchem.Mol):
            Chem.MolToPDBFile(x, temp_path)
            return temp_path
        if isinstance(x, str):
            if os.path.exists(x):
                *_, _format = x.rsplit(".")
                if _format.lower() == "pdb":
                    _ = Chem.MolFromPDBFile(x, sanitize=False)
                else:
                    _ = SSUDock.LoadLigand(x, native=False)  # TODO fix here
                    _ = Chem.MolToPDBBlock(_)
                Chem.MolToPDBFile(_, temp_path)
                return temp_path
            else:
                print(
                    "Not able to parse and create ligand file."
                    "Invalid File Path"
                )
    if clean:
        clean(os.path.dirname(temp_path))


def molecule_identifier(content, AA):
    if is_it_file(SSUDock.BASE_DIR, content):
        content = open(content)
        content = content.readlines()

    for line in content:
        if line.startswith("ATOM"):
            if (
                line[17:20] in AA
                or line.split()[-1] == "PROA"
                or line[21:22].strip() == "P"
            ):
                return "protein"
            elif line.split()[-1] == "MEMB" or line[21:22].strip() == "M":
                return "lipid"
            elif (
                line.split()[-1] == "TIP3"
                or line[17:20] == "HOH"
                or line[21:22].strip() == "T"
            ):
                return "water"
            elif (
                line.split()[-1] == "HETA"
                or line[72:76].strip() == "HETA"
                or line[17:20] == "LIG"
            ):
                return "ligand"
            else:
                return "ions"
        elif line.startswith("HETATM"):
            if line[17:20] == "HOH":
                return "water"
            elif len(line[17:20].strip()) < 3:
                return "ion"
            elif len(line[17:20].strip()) == 3:
                return "ligand"


def is_it_file(BASE_DIR, path):
    try:
        if os.path.exists(path):
            return True
        elif os.path.exists(os.path.join(BASE_DIR, path)):
            return True
        elif (
            isinstance(path, list) and len(
                path < 2) and os.path.exists(path[0])
        ):
            return True
        else:
            return False
    except Exception:
        return False


def clean(tree):
    try:
        shutil.rmtree(tree)
    except Exception:
        # print(er)
        pass


def main():
    mol = SSUDock("DATA")
    R = mol.LoadPDB(
        "/home/lab09/PACKAGES/ssudock-dev/notebooks/DATA/3eml_eqEND_a2a.pdb"
    )
    mol.MakeComplex(R[0], R[4])
    mol.ShowComplex()
    mol.ShowData
    mol.monitor()
    matrix = mol.rmsd_matrix()
    select(matrix, 1)
    console.print(select(matrix, "top"))


if __name__ == "__main__":
    print("Use as module")


Last updated: 28-Sept-2021 10:34:51
Use as module


Need cleaning after testing, run this function to clean all the artifacts that are created in process.

In [ ]:
#export 
def clean_all(*args):

    commands = [
        "./DATA/temp",
        " ./DATA/jobs",
        "./DATA/smina",
        "./DATA/poses",
        "./DATA/result",
        "./DATA/Receptor*",
        "./DATA/lipid*",
        "./DATA/ligand*",
        "./DATA/*log*",
        "./DATA/*output*",
    ]

    for arg in args:
        commands.append(arg)
    for command in commands:
        print(
            f"Cleaning.. {command.rsplit('/')[-1]:10} ✔️\n",
            sep="",
            end="",
            flush=True,
        )
        clean_command = f"rm -rf {command}"
        subprocess.run(clean_command, shell=True)


In [ ]:
mol = SSUDock()

Project Root set at /home/lab09/DEV/nbdev/ssu_dock/DATA

In [ ]:
mol.ProjectTree()

Supported File Format :{'pickle', 'pdbqt', 'excel', 'pdb', 'txt', 'sdf', 'csv'}

============Details of files====================

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ File Type                                ┃ Total Files ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────────────────────────────────────┴─────────────┘

In [ ]:
clean_all()

Cleaning.. temp       ✔️
Cleaning.. jobs       ✔️
Cleaning.. smina      ✔️
Cleaning.. poses      ✔️
Cleaning.. result     ✔️
Cleaning.. Receptor*  ✔️
Cleaning.. lipid*     ✔️
Cleaning.. ligand*    ✔️
Cleaning.. *log*      ✔️
Cleaning.. *output*   ✔️
